In [ ]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.stats import mode
from scipy.integrate import cumulative_trapezoid
from scipy.signal import correlate
import json
%config Completer.use_jedi = False  # Fixes autocomplete issues
%config InlineBackend.figure_format = 'retina'  # Improves plot resolution

import gc # garbage collector for removing large variables from memory instantly 
import importlib #for force updating changed packages 

#import harp
import harp_resources.process
import harp_resources.utils
from harp_resources import process, utils # Reassign to maintain direct references for force updating 
#from sleap import load_and_process as lp

In [ ]:
common_resampled_rate = 1000 #in Hz
event_name = "Apply halt: 2s" #event to align data to, can make into list in the future?

#Cohort 1 visual mismatch THIS
rawdata_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/Visual_mismatch_day3/B6J2717-2024-12-10T12-17-03')
data_path = rawdata_path.parent / f"{rawdata_path.name}_processedData/downsampled_data"
save_path = rawdata_path.parent / f"{rawdata_path.name}_processedData"
session_name = "_".join(data_path.parts[-2:])

# # Load all parquet files from the directory and assign to individual variables
# for file in data_path.glob("*.parquet"):
#     var_name = file.stem
#     if var_name == "session_settings":
#         session_settings = pd.read_parquet(file, engine="pyarrow")
#         session_settings["metadata"] = session_settings["metadata"].apply(process.safe_from_json)
#         print(f"Loaded {var_name} parquet file with special treatment")
#     else:
#         globals()[var_name] = pd.read_parquet(file, engine="pyarrow")
#         print(f"Loaded {var_name} parquet file")

# print(f"✅ Finished loading all parquet files from {data_path}")

photometry_tracking_encoder_data = pd.read_parquet(data_path / "photometry_tracking_encoder_data.parquet", engine="pyarrow")
camera_photodiode_data = pd.read_parquet(data_path / "camera_photodiode_data.parquet", engine="pyarrow")
experiment_events = pd.read_parquet(data_path / "experiment_events.parquet", engine="pyarrow")
photometry_info = pd.read_parquet(data_path / "photometry_info.parquet", engine="pyarrow")
session_settings = pd.read_parquet(data_path / "session_settings.parquet", engine="pyarrow")
session_settings["metadata"] = session_settings["metadata"].apply(process.safe_from_json)

print(f"✅ Finished loading all parquet files")


# # Load all parquet files from the directory and assign to individual variables
# loaded_vars = []  # Track loaded variable names
# for file in data_path.glob("*.parquet"):
#     var_name = file.stem
#     if var_name == "session_settings":
#         session_settings = pd.read_parquet(file, engine="pyarrow")
#         session_settings["metadata"] = session_settings["metadata"].apply(process.safe_from_json)
#         globals()[var_name] = session_settings
#         print(f"Loaded {var_name} parquet file with special treatment")
#     else:
#         globals()[var_name] = pd.read_parquet(file, engine="pyarrow")
#         loaded_vars.append(var_name)  # Track loaded variable names
#         print(f"Loaded {var_name} parquet file")

# print(f"✅ Finished loading all parquet files from {data_path}")

# # 🚀 Explicitly declare variable names so the linter recognizes them in pylance 
# photometry_tracking_encoder_data: pd.DataFrame = globals()["photometry_tracking_encoder_data"]
# camera_photodiode_data = pd.DataFrame = globals()["camera_photodiode_data"]
# experiment_events: pd.DataFrame = globals()["experiment_events"]
# photometry_info = pd.DataFrame = globals()["photometry_info"]
# session_settings: pd.DataFrame = globals()["session_settings"]



In [ ]:
photometry_tracking_encoder_data.info()


In [ ]:
#---------------------------------------------------
# PLOT FIGURE to ascertain everything is well loaded
#---------------------------------------------------
photodiode_halts = process.calculate_photodiode_falling_edges(camera_photodiode_data[['Photodiode']], experiment_events, event_name)
nearest_indices = photometry_tracking_encoder_data.index.get_indexer(photodiode_halts, method='nearest')
photodiode_halts = photometry_tracking_encoder_data.index[nearest_indices]
process.plot_figure_1(photometry_tracking_encoder_data, session_name, save_path, common_resampled_rate, photodiode_halts, save_figure = False, show_figure = True, downsample_factor=50)

In [ ]:
#-----------------------------------------
# Get Block times and durations
#-----------------------------------------

block_events = experiment_events[
    experiment_events["Event"].str.contains("block started|Block timer elapsed", case=False, na=False)
].copy()  # Use .copy() to avoid SettingWithCopyWarning
block_events["Time Difference"] = block_events.index.to_series().diff().dt.total_seconds()
block_events = block_events.drop(columns=['Seconds'])

# Print results
print(block_events)


In [ ]:
# Force reload the modules
importlib.reload(harp_resources.process)
importlib.reload(harp_resources.utils)
# Reassign after reloading to ensure updated references
process = harp_resources.process
utils = harp_resources.utils

NEXT - get event triggered averages of all relevant data and reproduce figure below 
**THIS IS NOT WORKING, SUGGEST 1. downsample photodiode signal and well and save togerther with dataset.**

NEXT - think about / run some correlation analysis 

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define columns to plot
columns_to_plot = ["Velocity_0X", "Velocity_0Y", "z_470", "z_560", "Encoder"]

# Create an empty DataFrame to store aligned data
aligned_data = []

# Loop through each photodiode halt event
for halt_time in photodiode_halts:
    # Extract data in the [-1s, +5s] window
    window_data = photometry_tracking_encoder_data.loc[
        (photometry_tracking_encoder_data.index >= halt_time - pd.Timedelta(seconds=1)) &
        (photometry_tracking_encoder_data.index <= halt_time + pd.Timedelta(seconds=5))
    ].copy()

    # Compute time relative to halt
    window_data["Time (s)"] = (window_data.index - halt_time).total_seconds()
    
    # Add event identifier
    window_data["Halt Time"] = halt_time

    # Store aligned data
    aligned_data.append(window_data)

# Concatenate all windows
aligned_df = pd.concat(aligned_data)

# Create subplots with shared x-axis
fig = make_subplots(rows=len(columns_to_plot), cols=1, shared_xaxes=True, subplot_titles=columns_to_plot)

# Add traces for each column
for i, col in enumerate(columns_to_plot, start=1):
    fig.add_trace(go.Scatter(
        x=aligned_df["Time (s)"],
        y=aligned_df[col],
        mode="lines",
        name=col
    ), row=i, col=1)

# Update layout
fig.update_layout(
    title="Time-Aligned Signal Responses Around Photodiode Halts",
    xaxis_title="Time (s) relative to halt",
    height=1200,  # Adjust height for better visibility
    template="plotly_white"
)

# Show the plot
fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define columns to plot
columns_to_plot = ["Position_0X", "Position_0Y", "z_470", "z_560", "Encoder"]

# Extract halt times from experiment_events where the event is "Applying halt: 2s"
halt_times = experiment_events[experiment_events["Event"] == "Applying halt: 2s"].index

# Create an empty DataFrame to store aligned data
aligned_data = []

# Loop through each halt event
for halt_time in halt_times:
    # Extract data in the [-1s, +5s] window
    window_data = photometry_tracking_encoder_data.loc[
        (photometry_tracking_encoder_data.index >= halt_time - pd.Timedelta(seconds=1)) &
        (photometry_tracking_encoder_data.index <= halt_time + pd.Timedelta(seconds=5))
    ].copy()

    # Compute time relative to halt
    window_data["Time (s)"] = (window_data.index - halt_time).total_seconds()
    
    # Add event identifier
    window_data["Halt Time"] = halt_time

    # Store aligned data
    aligned_data.append(window_data)

# Concatenate all windows
aligned_df = pd.concat(aligned_data)

# Create subplots with shared x-axis
fig = make_subplots(rows=len(columns_to_plot), cols=1, shared_xaxes=True, subplot_titles=columns_to_plot)

# Add traces for each column
for i, col in enumerate(columns_to_plot, start=1):
    fig.add_trace(go.Scatter(
        x=aligned_df["Time (s)"],
        y=aligned_df[col],
        mode="lines",
        name=col
    ), row=i, col=1)

# Update layout
fig.update_layout(
    title="Time-Aligned Signal Responses Around 'Applying halt: 2s' Events",
    xaxis_title="Time (s) relative to halt",
    height=1200,  # Adjust height for better visibility
    template="plotly_white"
)

# Show the plot
fig.show()


In [ ]:
pympler_memory_df = utils.get_pympler_memory_usage(top_n=10)
